In [35]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import json
from sklearn.preprocessing import MinMaxScaler
import joblib

## Загрузка данных

In [8]:
data = pd.read_csv('./data/amine_treatment-final.csv', on_bad_lines='skip')
data

,"feed_gas temperature, C","feed_gas pressure, kPa","feed_gas mass flow, kg/h",feed_gas molecular weight,"feed_gas Mass density, kg/m3","feed_gas actual liquid flow, m3/h","feed_gas actual vapour flow, m3/h",feed_gas CO2 mol frac,feed_gas Methane mol frac,feed_gas Ethane mol frac,...,degased_amine Propane mol frac,degased_amine i-Butane mol frac,degased_amine n-Butane mol frac,degased_amine i-Pentane mol frac,degased_amine n-Pentane mol frac,degased_amine H2S mol frac,degased_amine H2O mol frac,degased_amine MDEAmine mol frac,degased_amine H2S ppm,degased_amine CO2 ppm
0,52.0,6701.325,259647.0,24.668523,80.705482,0.000000,3217.216383,0.031875,0.633122,0.192729,...,0.000004,1.979803e-06,6.785930e-06,3.039502e-08,1.629357e-07,0.014838,0.910916,0.062270,22521.880092,23245.607009
1,40.0,6701.325,63853.0,23.411939,79.292250,0.000000,805.286774,0.039520,0.681947,0.179992,...,0.000006,1.099164e-07,7.886635e-07,2.425456e-06,2.679624e-08,0.010470,0.896612,0.077190,15891.219056,30468.484150
2,46.0,6701.325,15928.0,27.803853,108.358884,0.000000,146.993024,0.027859,0.577870,0.163095,...,0.000018,5.179434e-05,2.863001e-05,4.721545e-06,1.359401e-06,0.001284,0.860268,0.132676,1949.372531,10648.306282
3,36.0,6701.325,150276.0,22.053344,74.092459,0.000000,2028.222609,0.005420,0.856717,0.000249,...,0.000039,8.734358e-06,3.706189e-05,5.426945e-05,3.949909e-06,0.001233,0.809722,0.184359,1871.528366,8438.736973
4,12.0,6701.325,52691.0,24.334510,114.182302,0.002870,461.460940,0.027297,0.694634,0.091063,...,0.000100,7.719691e-05,2.661312e-05,1.351350e-05,6.619552e-06,0.003300,0.788340,0.200715,5008.207346,14011.150105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512795,18.0,6701.325,278813.0,27.267819,147.437736,0.239690,1890.816202,0.036621,0.666807,0.042159,...,0.000053,1.226987e-04,3.336251e-05,4.145371e-06,8.510407e-06,0.002101,0.796328,0.187149,3188.443718,27411.897443
512796,33.0,6701.325,72612.0,26.629816,112.530303,0.000000,645.266193,0.025383,0.605142,0.150361,...,0.000049,8.271254e-05,1.199104e-05,1.810930e-06,4.749955e-06,0.003784,0.826966,0.162727,5742.847844,11941.343733
512797,17.0,6701.325,272718.0,26.290335,135.390018,0.133398,2014.180541,0.039356,0.606850,0.167318,...,0.000037,2.874297e-05,3.343060e-06,1.045919e-05,2.187381e-06,0.045335,0.725301,0.179546,68812.636471,97071.672175
512798,59.0,6701.325,75970.0,24.231673,73.727565,0.000000,1030.415146,0.037951,0.732049,0.086286,...,0.000013,1.782460e-05,8.286743e-06,5.417825e-06,2.393693e-07,0.002551,0.877567,0.106280,3871.441495,26287.366187


## Модель для определения концентрации H2S в очищенном газе

In [59]:
mini_data = data.sample(frac=0.35)
len(mini_data)

179480

In [64]:
h2s_conc = mini_data[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac',
    'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
    'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac',
    'feed_gas H2S mol frac', 'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac',
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
    'lean_amine MDEAmine mol frac', 'sweet_gas H2S mol frac'
]]

X = h2s_conc[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac',
    'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
    'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac',
    'feed_gas H2S mol frac', 'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac',
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
    'lean_amine MDEAmine mol frac'
]]

y = h2s_conc[['sweet_gas H2S mol frac']]

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), [    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac',
    'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
    'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac',
    'feed_gas H2S mol frac', 'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac',
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
    'lean_amine MDEAmine mol frac'
                     ])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_norm = ct.transform(X_train)
X_test_norm = ct.transform(X_test)
print(X_test_norm.shape)
print(y_test)

(35896, 26)
        sweet_gas H2S mol frac
114688                0.016547
247022                0.000015
188811                0.000007
396255                0.000005
485979                0.000005
...                        ...
127839                0.000003
27580                 0.000005
26859                 0.000245
423389                0.000002
398504                0.000005

[35896 rows x 1 columns]


In [65]:
# scaler = MinMaxScaler()

# X_train_norm = scaler.fit_transform(X_train)
# X_test_norm = scaler.fit_transform(X_test)

# # Сохранение scaler
# joblib.dump(scaler, 'scaler.pkl')

# # Загрузка scaler
# loaded_scaler = joblib.load('scaler.pkl')

# # Преобразование данных в Node.js
# scaler_params = {'min': loaded_scaler.data_min_.tolist(), 'max': loaded_scaler.data_max_.tolist()}
# scaler_params_json = json.dumps(scaler_params)
# # передача scaler_params_json в Node.js
# with open('scaler_params_sweet_gas_h2s.json', 'w') as json_file:
#     json_file.write(scaler_params_json)

In [66]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
feed_streams_physic_model = tf.keras.Sequential([
    tf.keras.layers.Dense(26, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 2. Compile the model
feed_streams_physic_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# 3. Fit the model
feed_streams_physic_model.fit(X_train_norm, y_train, epochs=10)

Epoch 1/10
4487/4487 [==============================] - 14s 3ms/step - loss: 0.0012 - mae: 0.0012
Epoch 2/10
4487/4487 [==============================] - 13s 3ms/step - loss: 5.3170e-04 - mae: 5.3170e-04
Epoch 3/10
4487/4487 [==============================] - 12s 3ms/step - loss: 4.2485e-04 - mae: 4.2485e-04
Epoch 4/10
4487/4487 [==============================] - 10s 2ms/step - loss: 3.6656e-04 - mae: 3.6656e-04
Epoch 5/10
4487/4487 [==============================] - 14s 3ms/step - loss: 3.2802e-04 - mae: 3.2802e-04
Epoch 6/10
4487/4487 [==============================] - 15s 3ms/step - loss: 3.0953e-04 - mae: 3.0953e-04
Epoch 7/10
4487/4487 [==============================] - 13s 3ms/step - loss: 2.9329e-04 - mae: 2.9329e-04
Epoch 8/10
4487/4487 [==============================] - 14s 3ms/step - loss: 2.7915e-04 - mae: 2.7915e-04
Epoch 9/10
4487/4487 [==============================] - 13s 3ms/step - loss: 2.7656e-04 - mae: 2.7656e-04
Epoch 10/10
4487/4487 [==============================]

In [63]:
y_pred = feed_streams_physic_model.predict(X_test_norm)
y_pred

1122/1122 [==============================] - 3s 2ms/step


array([[ 6.588122e-03],
       [-6.948076e-05],
       [-6.948076e-05],
       ...,
       [ 4.014152e-04],
       [-6.948076e-05],
       [-6.948076e-05]], dtype=float32)